In [2]:
import orjson

from conda_oci_utils import encode_conda_dist_to_oci_dist, is_valid_oci_dist, is_valid_conda_dist
from conda_forge_metadata.repodata import fetch_repodata, all_labels

In [4]:
import tqdm

labels = all_labels(use_remote_cache=True)

max_len = 0
for label in tqdm.tqdm(labels, ncols=80):
    # tqdm.tqdm.write(f"Checking label: {label}")
    for subdir in [
        "linux-64",
        "osx-64",
        "noarch",
        "win-64",
        "linux-aarch64",
        "osx-arm64",
        "linux-ppc64le",
    ]:
        rd = fetch_repodata([subdir], label=label)[0]
        with open(rd) as f:
            rd = orjson.loads(f.read())

        for key in ["packages", "packages.conda"]:
            for dist in rd[key]:
                if not is_valid_conda_dist(dist):
                    tqdm.tqdm.write(f"invalid conda dist: {dist} (label {label})")
                oci_dist = encode_conda_dist_to_oci_dist(dist, channel="conda-forge", subdir=subdir)
                if not is_valid_oci_dist(oci_dist):
                    tqdm.tqdm.write(f"invalid oci dist: {oci_dist} (label: {label})")
                else:
                    if len(oci_dist) > max_len:
                        tqdm.tqdm.write(f"current maximum oci length: {len(oci_dist)}")
                    max_len = max(max_len, len(oci_dist))
                    # tqdm.tqdm.write(f"valid oci dist: {oci_dist} (label: {label})")


  7%|██▊                                      | 25/357 [00:00<00:02, 145.63it/s]

current maximum oci length: 48
current maximum oci length: 58
current maximum oci length: 62
current maximum oci length: 69
current maximum oci length: 74
current maximum oci length: 75
current maximum oci length: 77
current maximum oci length: 78
current maximum oci length: 112
current maximum oci length: 117
invalid conda dist: pyside2-2.0.0~alpha0-py27_0.tar.bz2 (label broken-and-invalid)
invalid oci dist: conda-forge/linux-64/cpyside2:2.0.0~alpha0-py27__0 (label: broken-and-invalid)
invalid conda dist: pyside2-2.0.0~alpha0-py35_0.tar.bz2 (label broken-and-invalid)
invalid oci dist: conda-forge/linux-64/cpyside2:2.0.0~alpha0-py35__0 (label: broken-and-invalid)
invalid conda dist: pyside2-2.0.0~alpha0-py36_0.tar.bz2 (label broken-and-invalid)
invalid oci dist: conda-forge/linux-64/cpyside2:2.0.0~alpha0-py36__0 (label: broken-and-invalid)
invalid conda dist: pyside2-2.0.0~alpha0-py27_0.tar.bz2 (label broken-and-invalid)
invalid oci dist: conda-forge/osx-64/cpyside2:2.0.0~alpha0-py27__

100%|█████████████████████████████████████████| 357/357 [11:13<00:00,  1.89s/it]
